TextRank

In [1]:
%%bash
apt-get update
apt-get install g++ openjdk-8-jdk python-dev python3-dev
pip3 install JPype1
pip3 install konlpy

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:14 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:15 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic

In [2]:
%env JAVA_HOME "/usr/lib/jvm/java-8-openjdk-amd64"

env: JAVA_HOME="/usr/lib/jvm/java-8-openjdk-amd64"


In [3]:
%%bash
bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)
pip3 install /tmp/mecab-python-0.996

mecab-ko is already installed
mecab-ko-dic is already installed
mecab-python is already installed
Done.
Processing /tmp/mecab-python-0.996
  Created wheel for mecab-python: filename=mecab_python-0.996_ko_0.9.2-cp37-cp37m-linux_x86_64.whl size=141812 sha256=731c0898758306e3e57be50fdd0fe8223ad58375585edce17333118727722cc1
  Stored in directory: /root/.cache/pip/wheels/40/7b/9f/2922869bef86c3354ae7034f7a3647c573ee1997c2dad0290a
Failed to build mecab-python
  Attempting uninstall: mecab-python
    Found existing installation: mecab-python 0.996-ko-0.9.2
    Uninstalling mecab-python-0.996-ko-0.9.2:
      Successfully uninstalled mecab-python-0.996-ko-0.9.2
    Running setup.py install for mecab-python: started
    Running setup.py install for mecab-python: finished with status 'done'


  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  DEPRECATION: mecab-python was installed using the legacy 'setup.py install' method, because a wheel could not be built for it. A possible replacement is to fix the wheel build issue reported above. You can find discussion regarding this at https://github.com/pypa/pip/issues/8368.


In [4]:
import numpy as np
import pandas as pd

import json
from pandas.io.json import json_normalize

import re
from tqdm.notebook import tqdm

import warnings
warnings.filterwarnings('ignore')

In [5]:
# train = pd.read_json('/content/drive/MyDrive/DACON_LG/data/train.json', lines=True)
# test = pd.read_json('/content/drive/MyDrive/DACON_LG/data/test.json', lines=True)

# print("train set :", train.shape)
# print("test set :", test.shape)

In [6]:
import os

DIR = "/content/drive/MyDrive/DACON_LG/data/"
TRAIN_SOURCE = os.path.join(DIR, "train.json")
TEST_SOURCE = os.path.join(DIR, "test.json")

In [7]:
with open(TRAIN_SOURCE) as f:
    TRAIN_DATA = json.loads(f.read())
    
with open(TEST_SOURCE) as f:
    TEST_DATA = json.loads(f.read())

In [8]:
train = pd.DataFrame(columns=['uid', 'title', 'region', 'context', 'summary'])
uid = 1000
for data in TRAIN_DATA:
    for agenda in data['context'].keys():
        context = ''
        for line in data['context'][agenda]:
            context += data['context'][agenda][line]
            context += ' '
        train.loc[uid, 'uid'] = uid
        train.loc[uid, 'title'] = data['title']
        train.loc[uid, 'region'] = data['region']
        train.loc[uid, 'context'] = context[:-1]
        train.loc[uid, 'summary'] = data['label'][agenda]['summary']
        uid += 1

test = pd.DataFrame(columns=['uid', 'title', 'region', 'context'])
uid = 2000
for data in TEST_DATA:
    for agenda in data['context'].keys():
        context = ''
        for line in data['context'][agenda]:
            context += data['context'][agenda][line]
            context += ' '
        test.loc[uid, 'uid'] = uid
        test.loc[uid, 'title'] = data['title']
        test.loc[uid, 'region'] = data['region']
        test.loc[uid, 'context'] = context[:-1]
        uid += 1

In [9]:
# train['total'] = train.title + ' ' + train.region + ' ' + train.context
# test['total'] = test.title + ' ' + test.region + ' ' + test.context

In [10]:
train

,uid,title,region,context,summary
1000,1000,제207회 완주군의회(임시회) 제 1 차 본회의회의록,완주,의석을 정돈하여 주시기 바랍니다. 성원이 되었으므로 제207회 완주군의회 임시회 제...,제207회 완주군의회 임시회 제1차 본회의 개의 선포.
1001,1001,제207회 완주군의회(임시회) 제 1 차 본회의회의록,완주,의사팀장 수고하셨습니다. 먼저 의사일정 제1항 제207회 완주군의회 임시회 회기 결...,제207회 완주군의회 임시회 회기는 8월 26일부터 9월 4일까지 10일간으로 가결됨.
1002,1002,제207회 완주군의회(임시회) 제 1 차 본회의회의록,완주,다음은 의사일정 제2항 제207회 완주군의회 임시회 회의록 서명의원 선출의 건을 상...,제207회 완주군의회 임시회 회의록 서명의원으로 최등원 의원과 박웅배 의원이 선출됨.
1003,1003,제207회 완주군의회(임시회) 제 1 차 본회의회의록,완주,다음은 의사일정 제3항 본회의 휴회의 건을 상정합니다. 상임의원회 의정활동을 위하여...,8월 27일부터 9월 3일까지 8일간 휴회가 가결됨. 제2차 본회의는 9월 4일 오...
1004,1004,제251회 완주군의회(제1차 정례회) 제1차 본 회 의 회 의 록,완주,의석을 정돈하여 주시기 바랍니다. 성원이 되었으므로 제251회 완주군의회 제1차 정...,제251회 완주군의회 제1차 정례회 제1차 본회의 개의 선포.
...,...,...,...,...,...
3989,3989,제327회 본회의 제2차(2020.08.26.),음성,회계과장입니다. 회계과 소관 2020년도 주요 현안사업을 보고드리겠습니다. 보고드릴...,음성읍행정복지센터 앞에 사유지 매입은 필요한 사업이기에 예산을 다시 반영해서라도 매...
3990,3990,제327회 본회의 제2차(2020.08.26.),음성,민원과장 김후식입니다. 민원과 소관 2020년도 주요 현안사업 추진상황을 보고드리겠...,공공시설물 전산화 작업시에 사유시설과 같이 협력해서 처리할 수 있도록 할 것. 지적...
3991,3991,제327회 본회의 제2차(2020.08.26.),음성,경제과장 박세덕입니다. 경제과 2020년 주요 현안사업에 대하여 보고드리겠습니다. ...,용계리 지역의 땅꺼짐 현상에 대해 군 차원의 대처를 위한 고민을 해줄 것. 생극면 ...
3992,3992,제327회 본회의 제2차(2020.08.26.),음성,농정과장 김기연입니다. 농정과 소관 2020년도 하반기 주요 현안사업을 보고드리겠습...,음성군에서 생산된 쌀이 진천으로 수매하는 현상이 발생하고 있는데 음성군 브랜드를 활...


In [11]:
test

,uid,title,region,context
2000,2000,제235회 본회의 제1차(2012.06.21.),음성,의석을 정돈하여 주시기 바랍니다. 성원이 되었으므로 지금부터 음성군의회 제235회 ...
2001,2001,제235회 본회의 제1차(2012.06.21.),음성,"의사일정 제1항, 음성군의회 제235회 제1차 정례회 회기결정의 건을 상정합니다. ..."
2002,2002,제235회 본회의 제1차(2012.06.21.),음성,"의사일정 제2항, 회의록 서명의원 선출의 건을 상정합니다. 제235회 제1차 정례회..."
2003,2003,제235회 본회의 제1차(2012.06.21.),음성,"의사일정 제3항, 예산결산특별위원회 구성의 건을 상정합니다. 예산결산특별위원회 구성..."
2004,2004,제235회 본회의 제1차(2012.06.21.),음성,"의사일정 제4항, 환경분야 현지확인 특별위원회 구성결의안을 상정합니다. 대표발의하신..."
...,...,...,...,...
2501,2501,제174회 본회의 제2차(2006.10.16.),음성,"의사일정 제1항, 제174회 음성군의회 임시회 회기 연장의 건을 상정합니다. 제17..."
2502,2502,제174회 본회의 제2차(2006.10.16.),음성,"의사일정 제2항, 농협 축산물공판장 이전관련 현지확인 특별위원회 활동기간 연장의 건..."
2503,2503,제174회 본회의 제2차(2006.10.16.),음성,"의사일정 제3항, 음성군 대소도서관 관리운영 조례안을 상정합니다. 문화공보과장께서는..."
2504,2504,제174회 본회의 제2차(2006.10.16.),음성,"의사일정 제4항, 제4기 음성군 지역보건 의료계획안에 대해서 보건소장께서는 나오셔서..."


In [12]:
test['context']

2000    의석을 정돈하여 주시기 바랍니다. 성원이 되었으므로 지금부터 음성군의회 제235회 ...
2001    의사일정 제1항, 음성군의회 제235회 제1차 정례회 회기결정의 건을 상정합니다. ...
2002    의사일정 제2항, 회의록 서명의원 선출의 건을 상정합니다. 제235회 제1차 정례회...
2003    의사일정 제3항, 예산결산특별위원회 구성의 건을 상정합니다. 예산결산특별위원회 구성...
2004    의사일정 제4항, 환경분야 현지확인 특별위원회 구성결의안을 상정합니다. 대표발의하신...
                              ...                        
2501    의사일정 제1항, 제174회 음성군의회 임시회 회기 연장의 건을 상정합니다. 제17...
2502    의사일정 제2항, 농협 축산물공판장 이전관련 현지확인 특별위원회 활동기간 연장의 건...
2503    의사일정 제3항, 음성군 대소도서관 관리운영 조례안을 상정합니다. 문화공보과장께서는...
2504    의사일정 제4항, 제4기 음성군 지역보건 의료계획안에 대해서 보건소장께서는 나오셔서...
2505    의사일정 제5항, 휴회의 건을 상정합니다. 제174회 임시회 휴회의 건은 농협축산물...
Name: context, Length: 506, dtype: object

In [13]:
!pip install git+https://github.com/lovit/textrank.git

  Cloning https://github.com/lovit/textrank.git to /tmp/pip-req-build-ie_1m5kt
  Running command git clone -q https://github.com/lovit/textrank.git /tmp/pip-req-build-ie_1m5kt


In [14]:
from konlpy.tag import Komoran
from textrank import KeysentenceSummarizer
import textrank

komoran = Komoran()

def komoran_tokenizer(sent):
    words = komoran.pos(sent, join=True)
    words = [w for w in words if ('/NN' in w or '/XR' in w or '/VA' in w or '/VV' in w)]
    return words

summarizer = KeysentenceSummarizer(
    tokenize = komoran_tokenizer,
    min_sim = 0.3,
    verbose = False
)

In [15]:
summary = []

for text in tqdm(test['context']):
    keysents = summarizer.summarize(text, topk=3)
    summary.append(keysents)

  0%|          | 0/506 [00:00<?, ?it/s]

java.lang.NullPointerException: ignored

In [ ]:
extractive = []

for n in range(test.shape[0]):
    result = summary[n][0][2] + '\n' + summary[n][1][2] + '\n' + summary[n][2][2]
    extractive.append(result)

submission = pd.read_csv("/content/drive/MyDrive/DACON_LG/data/dacon_baseline.csv")
submission['summary'] = extractive
submission.to_csv("/content/drive/MyDrive/DACON_LG/TextRank.csv", index=False)
submission